In [188]:
import random
import csv
from typing import Optional
import cachetools
from math import inf
from functools import partial
from ortools.algorithms import pywrapknapsack_solver

random.seed(42)

RAM_SIZE = 20 * 1024 * 1024 
DISK_SIZE = 20 * 1024 * 1024 - int(0.5 * 1024 * 1024)

In [189]:
page_views={}
with open("pageviews.csv", "r") as f:
    reader = csv.DictReader(f)
    for line in reader:
        page_views[line["article"].replace(" ","_")] = int(line["views"])

In [190]:
page_sizes = {}
with open("page_sizes.csv","r") as f:
    reader = csv.DictReader(f)
    for line in reader:
        page_sizes[line["Page"]] = int(line["Size"]) # Note: Size is uncompressed
print(f"Total = {sum(page_sizes.values())} bytes")

Total = 502102865 bytes


In [191]:
single_page_compressed_size = {}
with open("brotli_sizes.csv","r") as f:
    reader = csv.DictReader(f)
    for line in reader:
        single_page_compressed_size[line["Page"]] = int(line["Size"]) 
print(f"Total = {sum(single_page_compressed_size.values())} bytes")

Total = 67609330 bytes


In [192]:
POPULAR_FILES = set()
total = 0
for file in sorted(page_views, key=lambda pg: page_views[pg], reverse=True):
    size = single_page_compressed_size[file]
    if total + size <= DISK_SIZE:
        total += size 
        POPULAR_FILES.add(file)
print(len(POPULAR_FILES))

297


In [193]:
SMALLEST_FILES = set()
total = 0
for file, size in sorted(single_page_compressed_size.items(), key=lambda x: x[1]):
    if total + size  <= DISK_SIZE:
        total += size 
        SMALLEST_FILES.add(file)
print(len(SMALLEST_FILES))

567


In [194]:
LARGEST_FILES = set()
total = 0
for file, size in sorted(single_page_compressed_size.items(), key=lambda x: x[1], reverse=True):
    if total + size <= DISK_SIZE:
        total += size 
        LARGEST_FILES.add(file)
print(len(LARGEST_FILES))

130


In [195]:
# Max 7z compression = int(page_sizes[file] / 10.75)
values = [page_views[page] for page in sorted(page_views)]
weights = [[single_page_compressed_size[page] for page in sorted(page_views)]] 
capacity = [DISK_SIZE]

solver = pywrapknapsack_solver.KnapsackSolver(
    pywrapknapsack_solver.KnapsackSolver.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER, "Disk"
)

solver.Init(values,weights,capacity)
ans = solver.Solve()

DISK_KNAPSACK = {
    file
    for i, file in enumerate(sorted(page_views))
    if solver.BestSolutionContains(i)
}
print(len(DISK_KNAPSACK))

512


In [196]:
ZIP_FILES = DISK_KNAPSACK

In [197]:
RAM_SMALLEST = set()
total = 0
for file, size in sorted(single_page_compressed_size.items(), key=lambda x: x[1]):
    if file in ZIP_FILES: continue
    if total + size <= RAM_SIZE:
        total += size
        RAM_SMALLEST.add(file)
print(len(RAM_SMALLEST))


293


In [198]:
RAM_POPULAR = set()
total = 0
for file, size in sorted(single_page_compressed_size.items(), key=lambda x: page_views[x[0]], reverse=True):
    if file in ZIP_FILES: continue
    if total + size <= RAM_SIZE:
        total += size
        RAM_POPULAR.add(file)
print(len(RAM_POPULAR))

181


In [199]:
remaining = list(page for page in sorted(page_views)
                 if page not in DISK_KNAPSACK)
values = [page_views[page] for page in remaining]
weights = [[single_page_compressed_size[page] for page in remaining]]
capacity = [RAM_SIZE]

solver = pywrapknapsack_solver.KnapsackSolver(
    pywrapknapsack_solver.KnapsackSolver.
    KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER, "RAM1")

solver.Init(values, weights, capacity)
ans = solver.Solve()

RAM_KNAPSACK = {
    file
    for i, file in enumerate(remaining) if solver.BestSolutionContains(i)
}
print(len(RAM_KNAPSACK))

271


In [200]:
class CacheStrategy:

    def __init__(self, max_size=0) -> None:
        self.max_size = max_size
        self.cache = {}
        self.size = 0
        self.total_hits = 0
        self.total_requests = 0

    def get(self, key) -> Optional[str]:
        self.total_requests += 1

        on_disk = key in ZIP_FILES 
        on_ram = key in self.cache
        self.total_hits += on_disk or on_ram

        if on_disk or on_ram:
            return True
        else:
            self.set(key, single_page_compressed_size[key])
            return None

    def set(self, key, size) -> None:
        raise NotImplementedError

In [201]:
class NoStrategy(CacheStrategy):

    def set(self, key, value) -> None:
        pass


class CacheToolStrategy(CacheStrategy):

    def __init__(self, max_size=inf, cachetools_strategy=None) -> None:
        super().__init__()
        self.max_size = max_size
        self.cache = cachetools_strategy(maxsize=max_size)

    def set(self, key, size) -> None:
        while self.size + size > self.max_size and len(self.cache):
            self.size -= self.cache.popitem()[1]
        if self.size + size <= self.max_size:
            self.cache[key] = size
            self.size += size


class PreloadCache(CacheStrategy):

    def __init__(self, max_size=inf, preload_files=None) -> None:
        super().__init__(max_size)
        self.cache = {k: -1 for k in preload_files}

    def set(self, key, size) -> None:
        pass


class PreloadWithEviction(CacheStrategy):

    def __init__(self,
                 max_size=inf,
                 preload_files=None,
                 cachetools_strategy=None) -> None:
        super().__init__(max_size)
        self.cache = cachetools_strategy(maxsize=max_size)
        for f in preload_files:
            self.cache[f] = single_page_compressed_size[f]
        self.size = sum(self.cache.values())

    def set(self, key, size) -> None:
        while self.size + size > self.max_size and len(self.cache):
            self.size -= self.cache.popitem()[1]
        if self.size + size <= self.max_size:
            self.cache[key] = size
            self.size += size

In [202]:
with_max_size = partial(CacheToolStrategy, max_size=RAM_SIZE)
strategies = {
    "No cache": NoStrategy(max_size=RAM_SIZE),
    "LRU": with_max_size(cachetools_strategy=cachetools.LRUCache),
    "LFU": with_max_size(cachetools_strategy=cachetools.LFUCache),
    "Preload Smallest": PreloadCache(max_size=0,preload_files=RAM_SMALLEST),
    "Preload Popular": PreloadCache(max_size=0,preload_files=RAM_POPULAR),
    "Preload Knapsack": PreloadCache(max_size=0,preload_files=RAM_KNAPSACK),
    "LFU Preload Smallest":PreloadWithEviction(max_size=RAM_SIZE,preload_files=RAM_SMALLEST,cachetools_strategy=cachetools.LFUCache),
    "LFU Preload Popular":PreloadWithEviction(max_size=RAM_SIZE,preload_files=RAM_POPULAR,cachetools_strategy=cachetools.LFUCache),
    "LRU Preload Knapsack":PreloadWithEviction(max_size=RAM_SIZE,preload_files=RAM_SMALLEST,cachetools_strategy=cachetools.LRUCache),
    "LFU Preload Knapsack":PreloadWithEviction(max_size=RAM_SIZE,preload_files=RAM_KNAPSACK,cachetools_strategy=cachetools.LFUCache),
    # "FIFO": with_max_size(cachetools_strategy=cachetools.FIFOCache),
    # "MRU": with_max_size(cachetools_strategy=cachetools.MRUCache),
    # "RR": with_max_size(cachetools_strategy=cachetools.RRCache),
}

In [203]:
SEQUENCE_SIZE = 10**6
articles = list(page_views.keys())
views = [page_views[article] for article in articles]
sequence = random.choices(population=articles, weights=views, k=SEQUENCE_SIZE)

for article in sequence:
    for strategy in strategies.values():
        strategy.get(article)

for name, obj in strategies.items():
    print(f"{name} hit rate: {obj.total_hits / obj.total_requests}")

No cache hit rate: 0.74943
LRU hit rate: 0.86311
LFU hit rate: 0.86311
Preload Smallest hit rate: 0.888167
Preload Popular hit rate: 0.876184
Preload Knapsack hit rate: 0.902507
LFU Preload Smallest hit rate: 0.887406
LFU Preload Popular hit rate: 0.875418
LRU Preload Knapsack hit rate: 0.863201
LFU Preload Knapsack hit rate: 0.901514


In [204]:
with open("disk_files.txt","w") as f:
    print(*sorted(DISK_KNAPSACK), sep="\n",file=f)
with open("preload_files.txt","w") as f:
    print(*sorted(RAM_KNAPSACK), sep="\n",file=f)